In [1]:
# Have to install janitor package to run. Not included in anaconda.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor

In [2]:
exec(open("../header.py").read())

# Import raw data

In [3]:
sample = False
header = Header(sample)

In [4]:
raw_crsp = pd.read_csv(header.clean_root("crsp.csv"), parse_dates = ['date'])
raw_rs = pd.read_csv(header.clean_root("russell.csv"), parse_dates = ['date'])

C:\Users\Andrew\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
raw_crsp.head(2)

,permno,date,ticker,comnam,tsymbol,hexcd,cusip,prc,vol,ret,...,shrout,numtrd,ewretd,mktcap,year,month,yrmo,bid_ask_spread,turnover,dollar_vol
0,10001,1995-05-01,EWST,ENERGY WEST INC,EWST,2,29274A10,8.250,400.0,0.1,...,2244.0,3.0,0.000588,18513.0,1995,5,199505,0.095238,0.021606,3300.0
1,10001,1995-05-02,EWST,ENERGY WEST INC,EWST,2,29274A10,7.875,0.0,-0.045455,...,2244.0,0.0,0.001771,17671.5,1995,5,199505,0.095238,0.000000,0.0


In [6]:
raw_rs.head(2)

,date,cusip,ticker,mktvalue,shares,russell1000,russell2000,name,r1000_wt,r2000_wt,year,month,yrmo
0,1996-01-31,00036110,AIR,NaN,15959,0,1,AAR CORP,0.0,0.0599,1996,1,199601
1,1996-01-31,00075210,ABCR,NaN,7983,0,1,ABC RAIL PRODUCTS CO,0.0,0.0304,1996,1,199601


# Problems with CUSIP
* Example: Citigroup
* Link: https://wrds-www.wharton.upenn.edu/pages/support/support-articles/mergent-fisd/linking-mergent-compustat-or-crsp/

In [7]:
print(f"Russell CUSIP for Citigroup: {raw_rs.loc[lambda x:(x.ticker == 'C'),'cusip'].unique()}")
print(f"CRSP CUSIP for Citigroup: {raw_crsp.loc[lambda x:(x.ticker == 'C'),'cusip'].unique()}")

Russell CUSIP for Citigroup: ['17119610' '17296710']
CRSP CUSIP for Citigroup: ['17119610' '17296710']


In [8]:
raw_crsp.loc[lambda x:(x.ticker == 'C')&(x.date >= '1998-11-12')&(x.date <= '1998-12-8'),['date','cusip']]

,date,cusip
765406,1998-11-12,17119610
6127667,1998-12-04,17296710
6127668,1998-12-07,17296710
6127669,1998-12-08,17296710


In [9]:
raw_rs.loc[lambda x:(x.ticker == 'C')&(x.date >= '1998-08-01')&(x.date <= '1999-02-01'),['date','cusip']]

,date,cusip
90868,1998-08-31,17119610
93828,1998-09-30,17119610
96765,1998-10-30,17119610
102601,1998-12-31,17296710
105494,1999-01-29,17296710


# Pre-merge cleaning

In [10]:
rs = raw_rs\
    .drop(['date', 'mktvalue', 'shares', 'year', 'month'], axis = 1)

In [11]:
crsp = raw_crsp

# Join on month and cleaned cusip

In [12]:
rs_companies = rs\
    .loc[:,['cusip']]\
    .drop_duplicates()

In [13]:
# Has to be left so can create rankings for companies that just entered the Russell indices.
# Or else if inner, drops the past market cap data for those companies
# So do the first join to filter down to companies that are in Russell at some point in time
crsp_join = crsp\
    .merge(rs_companies, how = "inner", on = ["cusip"])\
    .merge(rs, how = "left", on = ["yrmo", "cusip"])

In [14]:
crsp_join.head(3)

,permno,date,ticker_x,comnam,tsymbol,hexcd,cusip,prc,vol,ret,...,yrmo,bid_ask_spread,turnover,dollar_vol,ticker_y,russell1000,russell2000,name,r1000_wt,r2000_wt
0,10002,2002-05-15,BTFG,BANCTRUST FINANCIAL GROUP INC,BTFG,3,05978R10,11.55,1875.0,-8.7e-05,...,200205,0.017021,0.018555,21656.25,NaN,NaN,NaN,NaN,NaN,NaN
1,10002,2002-05-16,BTFG,BANCTRUST FINANCIAL GROUP INC,BTFG,3,05978R10,11.75,1500.0,0.017316,...,200205,0.008475,0.014591,17625.00,NaN,NaN,NaN,NaN,NaN,NaN
2,10002,2002-05-17,BTFG,BANCTRUST FINANCIAL GROUP INC,BTFG,3,05978R10,11.80,0.0,0.004255,...,200205,0.008475,0.000000,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
print(f"CRSP // Original: {crsp.shape[0]}, Merged: {crsp_join.shape[0]}, Ratio: {crsp_join.shape[0]/crsp.shape[0]:.2f}")
print(f"Russell // Original: {rs.shape[0]}")
print(f"Unique  companies // Pre CRSP: {len(crsp.cusip.unique())}, Russell: {len(rs.cusip.unique())}, Post: {len(crsp_join.cusip.unique())}")

CRSP // Original: 22198012, Merged: 10410679, Ratio: 0.47
Russell // Original: 386154
Unique  companies // Pre CRSP: 19076, Russell: 7775, Post: 7325


Missing ~400/7775 companies that should be in CRSP.

# Data integrity

In [16]:
crsp_join.columns

Index(['permno', 'date', 'ticker_x', 'comnam', 'tsymbol', 'hexcd', 'cusip',
       'prc', 'vol', 'ret', 'bid', 'ask', 'shrout', 'numtrd', 'ewretd',
       'mktcap', 'year', 'month', 'yrmo', 'bid_ask_spread', 'turnover',
       'dollar_vol', 'ticker_y', 'russell1000', 'russell2000', 'name',
       'r1000_wt', 'r2000_wt'],
      dtype='object')

In [17]:
# Check that cusip-date is still a unique identifier
assert crsp_join\
    .groupby(['date', 'cusip'], as_index = False)\
    .agg(counts = ('ticker_x', 'count'))\
    .loc[lambda x:x.counts > 1]\
    .shape[0] == 0

In [18]:
# Validate that cusip match is reasonable
# Even when tickers don't match, clearly seem like the same company based on company name
crsp_join\
    .loc[lambda x:~x.ticker_y.isna()]\
    .loc[lambda x:x.ticker_x != x.ticker_y, ['ticker_x', 'ticker_y', 'tsymbol', 'comnam', 'name']]\
    .drop_duplicates()\
    .head(10)

,ticker_x,ticker_y,tsymbol,comnam,name
24692,UICI,UCI,UICI,U I C I,UICI
27562,KIDE,KDE,KIDE,4 KIDS ENTERTAINMENT INC,4 KIDS ENTMT INC
36171,SNDT,SDS,SNDT,SUNGARD DATA SYSTEMS INC,SUNGARD DATA SYSTEMS
47429,OVWV,OV,OVWV,ONE VALLEY BANCORP INC,ONE VALLEY BANCORP/W
85461,XCED,XCEDD,XCEDD,WORLDWIDE XCEED GROUP INC,WORLDWIDE XCEED GROU
123768,AVM,AMAG,AVM,ADVANCED MAGNETICS INC,ADVANCED MAGNETICS INC
138341,TREN,TWK,TREN,TRENWICK GROUP INC,TRENWICK GROUP INC
153975,SKYW,SKYWE,SKYW,SKYWEST INC,SKYWEST INC
153990,SKYW,SKYWE,SKYWE,SKYWEST INC,SKYWEST INC
186862,SVRN,SOV,SVRN,SOVEREIGN BANCORP INC,SOVEREIGN BANCORP IN


In [19]:
# No invalid market cap, bid-ask spread, volume, turnover
assert np.sum(crsp_join.mktcap.isna()) == 0
assert np.sum(crsp_join.bid_ask_spread.isna()) == 0
assert np.sum(crsp_join.turnover.isna()) == 0
assert np.sum(crsp_join.dollar_vol.isna()) == 0

# Save cleaned datsets

In [20]:
crsp_join.to_csv(header.clean_root("crsp_russ.csv"), index = False)